# Telco Customer Churn Analysis

This Jupyter notebook works on analyzing the customer pattern of Telco. I will try to use the demographic and service usage to predict if the customer will leave or remain to use the same service by Telco. 

The data is taken from:
https://www.kaggle.com/blastchar/telco-customer-churn

Prepared by Shing Chi Leung at 14 May 2021

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

In [2]:
directory = "kaggle_telco/"
filename = "telco_churn.csv"

df = pd.read_csv(directory+filename)

## Basic Data Review

In [3]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


Then I will extract some key features about the dataframe. 

In [4]:
columns = df.columns
columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [5]:
size = df.size
size 

147903

In [6]:
print("The number of unique items in each column:")
for column in columns:
    print(column, df[column].nunique(), df[column].unique())

The number of unique items in each column:
customerID 7043 ['7590-VHVEG' '5575-GNVDE' '3668-QPYBK' ... '4801-JZAZL' '8361-LTMKD'
 '3186-AJIEK']
gender 2 ['Female' 'Male']
SeniorCitizen 2 [0 1]
Partner 2 ['Yes' 'No']
Dependents 2 ['No' 'Yes']
tenure 73 [ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26  0
 39]
PhoneService 2 ['No' 'Yes']
MultipleLines 3 ['No phone service' 'No' 'Yes']
InternetService 3 ['DSL' 'Fiber optic' 'No']
OnlineSecurity 3 ['No' 'Yes' 'No internet service']
OnlineBackup 3 ['Yes' 'No' 'No internet service']
DeviceProtection 3 ['No' 'Yes' 'No internet service']
TechSupport 3 ['No' 'Yes' 'No internet service']
StreamingTV 3 ['No' 'Yes' 'No internet service']
StreamingMovies 3 ['No' 'Yes' 'No internet service']
Contract 3 ['Month-to-month' 'One year' 'Two year']
PaperlessBilling 2 ['Yes' 'No']
PaymentMetho

The features of the dataframe is very clean. So what we need to do is to replace all the category into numbers so that it can be readily classified by the machine learning algorithm. 

## Data Cleaning

I will replace all columns according to their unique entries along the same column. 

In [7]:
def internet_tokenize(row, column):
    if row[column] == "No":
        return 0
    elif row[column] == "Yes":
        return 1
    else:
        return -1
    
internet_columns = ["OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies"]

for column in internet_columns:
    df[column] = df.apply(lambda x: internet_tokenize(x, column), axis=1)

In [8]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,0,...,0,0,0,0,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,1,...,1,0,0,0,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,1,...,0,0,0,0,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,1,...,1,1,0,0,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,0,...,0,0,0,0,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
def gender_tokenize(row, column):
    if row[column] == "Male":
        return 1
    else: 
        return 0
    
df["gender"] = df.apply(lambda x: gender_tokenize(x, "gender"), axis=1)

In [10]:
def binary_tokenize(row, column):
    if row[column] == "Yes":
        return 1
    else: 
        return 0
    
binary_columns = ["Partner", "Dependents", "PhoneService", "PaperlessBilling", "Churn"]

for column in binary_columns:
    df[column] = df.apply(lambda x: binary_tokenize(x, column), axis=1)

In [11]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,No phone service,DSL,0,...,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,No,DSL,1,...,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,No,DSL,1,...,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,No phone service,DSL,1,...,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,No,Fiber optic,0,...,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1


In [12]:
def multipleLines_tokenize(row, column):
    if row[column] == "Yes":
        return 1
    elif row[column] == "No":
        return 0
    else:
        return -1
    
df["MultipleLines"] = df.apply(lambda x: gender_tokenize(x, "MultipleLines"), axis=1)

In [13]:
df["InternetService"] = df["InternetService"].replace({"DSL":1, "Fiber optic":2, "No":0})
df["Contract"] = df["Contract"].replace({'Month-to-month':0, 'One year':1, 'Two year':2})
df["PaymentMethod"] = df["PaymentMethod"].replace(
    {"Electronic check":0, "Mailed check":1, "Bank transfer (automatic)":2, "Credit card (automatic)":3})

In [14]:
df.head(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,1,0,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,0,1,1,...,1,0,0,0,1,0,1,56.95,1889.5,0
2,3668-QPYBK,1,0,0,0,2,1,0,1,1,...,0,0,0,0,0,1,1,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,0,1,1,...,1,1,0,0,1,0,2,42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,0,2,0,...,0,0,0,0,0,1,0,70.70,151.65,1


In [15]:
def monthlyCharges_tokenize(row, column):
    if row[column] == "unknown":
        return 0
    else:
        return row[column]
    
df["MonthlyCharges"] = df.apply(lambda x: monthlyCharges_tokenize(x, "MonthlyCharges"), axis=1)

In [16]:
def totalCharges_tokenize(row, column):
    if row[column].isnumeric():
        return float(row[column])
    else:
        return 0

df["TotalCharges"] = df.apply(lambda x: totalCharges_tokenize(x, "TotalCharges"), axis=1)

In [17]:
# print("The number of unique items in each column:")
# for column in columns:
#     print(column, df[column].nunique(), df[column].unique())

df.dtypes

customerID           object
gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
OnlineBackup          int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
PaymentMethod         int64
MonthlyCharges      float64
TotalCharges        float64
Churn                 int64
dtype: object

Now all the string and object type data (except customerID which is not important for our prediction) is now all in numerical. We can proceed for machine learning phase. 

## Train/Test Data preparation

Now having prepared the dataframe to be appropriate for classification by sklearn, let us split the data into training and testing sets. 

In [19]:
data = df.to_numpy()

In [20]:
data.shape

(7043, 21)

In [21]:
X = data[:,1:20]
y = data[:,20]
y = y.astype('int')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print("train size", X_train.shape)
print("test size", X_test.shape)
print("Number of positive test case", sum(y_test))

train size (5634, 19)
test size (1409, 19)
Number of positive test case 373


We have successfully prepared our training and testing set. The typical negative case is about 25% of the whole testing set. This is not a small number so thus we do not need to 

## Classification by Single Decision Tree

Now let us build a binary decision tree. We will use the training set to train the tree and then test with the testing set. 

In [33]:
dtree = DecisionTreeClassifier()
clf = dtree.fit(X_train, y_train)

print("The importance of each feature: ")
for column, imp in zip(columns, dtree.feature_importances_):
    print("{:20}: {:.4}".format(column, imp))

The importance of each feature: 
customerID          : 0.02388
gender              : 0.01921
SeniorCitizen       : 0.03034
Partner             : 0.01504
Dependents          : 0.2017
tenure              : 0.006498
PhoneService        : 0.0
MultipleLines       : 0.04428
InternetService     : 0.02428
OnlineSecurity      : 0.02254
OnlineBackup        : 0.0198
DeviceProtection    : 0.01352
TechSupport         : 0.006439
StreamingTV         : 0.01164
StreamingMovies     : 0.1664
Contract            : 0.02901
PaperlessBilling    : 0.04789
PaymentMethod       : 0.3012
MonthlyCharges      : 0.01627


The result is actually surprising. Most of the parameters, such as using Phone Serice or Tech Support, are irrelevant to whether the customers churn or not. Instead, the most important one are:

1. Payment method
2. Dependents
3. Streaming movies

The number of dependents will genearlly refer to the number of children. I can imagine that when there are more people in the same apartment using the service, there is less insensitive to change it. Similarly, a good bandwidth will be important for streaming movies, which is a popular activity nowadays. And whether or not the Telco can provide a good/stable bandwidth for streaming movie will therefore be important for the customers to stay with this company. 

The payment method is the surprising part of the model fitting. It should be explored in details why some payment method has stronger tendency to churn. 

Then let us proceed to test the accuracy of the model. 

In [28]:
y_pred2 = dtree.predict(X_test)

confusion_matrix(y_pred, y_test)

array([[940, 192],
       [ 96, 181]], dtype=int64)

In [30]:
print("Classification Report: \n{}\n".format(classification_report(y_pred2, y_test)))

Classification Report: 
              precision    recall  f1-score   support

           0       0.80      0.80      0.80      1034
           1       0.46      0.45      0.45       375

    accuracy                           0.71      1409
   macro avg       0.63      0.63      0.63      1409
weighted avg       0.71      0.71      0.71      1409




In [31]:
roc_auc_score(y_pred2, y_test)

0.6285041908446164

In [26]:
print(dtree.get_depth())

25


From the classification report, we see that the accuracy of the model is not high. It has a competable recal rate for negative case. But it has a low precision rate for positive case. 

The above ROC_AUC score is only slight higher than 0.5 which means that a single decision tree is insufficient in providing an accurate prediction for our data. 

This means that a single forest will tend to predict wrongly customers who wants to churn. 

## Analysis by Random Forest

The decision tree classifier does not give a very good job that the precision and recall rate are not very high. Let us consider the random forest to see if more trees can help. Again, let us use the default setting first. 

In [22]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

print(rfc.score(X_train, y_train))

0.9980475683351083


In [23]:
y_pred = rfc.predict(X_test)

confusion_matrix(y_pred, y_test)

array([[940, 192],
       [ 96, 181]], dtype=int64)

In [24]:
print("Classification Report: \n{}\n".format(classification_report(y_pred, y_test)))

Classification Report: 
              precision    recall  f1-score   support

           0       0.91      0.83      0.87      1132
           1       0.49      0.65      0.56       277

    accuracy                           0.80      1409
   macro avg       0.70      0.74      0.71      1409
weighted avg       0.82      0.80      0.81      1409




In [27]:
roc_auc_score(y_pred, y_test)

0.7419091477337959

In contrast to the single decision tree, the decision forest provides a better prediction. It receives a higher accuracy for the testing set, as well as higher precision and recall rate. The ROC_AUC score is also significantly higher. 